In [7]:
%matplotlib inline

In [8]:
import numpy as np

In [35]:
def sparsematrix(n, p, slist, v=1.0, m=0.0):
    ''' generate (n, k) matrix that has [slist] non-zeros on every column'''
    M = np.zeros((n, p))
    for ix in range(p): # for every column
        sval = slist[ix]
        index = np.random.permutation(n)[:sval]
        values = v*np.random.randn(sval) + m
        M[index, ix] = values
    return M

In [121]:
# set hyperprameters
P = 150 # number of regions
N = 2 # number of subjects
T = 5 # number of samples per subject
Ka = 3 # number of known shared factors (networks) < p
Kc = 1 # number of new factors (networks)
Sa = [int(P/2)]*Ka #sparsity level for each factor < p
Sc = [int(P/5)]*Kc #sparsity level for each factor < p
snr = 100 # signal to noise ratio

# generate data

In [122]:
# meta parameters
cova = np.ones(Ka) # factor covariance of known samples
covc = np.ones(Kc) # factor covariance of new samples

# Generate group level
A = sparsematrix(P, Ka, Sa, 10.0/np.sqrt(P)) # core factors
B = np.random.multivariate_normal(np.zeros(Ka), np.diag(cova), (N, T)) # temporal weights for knwon factors (N, T, Ka)
C=0; D=0
if Kc>0:
    C = sparsematrix(P, Kc, Sc, 10.0/np.sqrt(P)) # new factors
    D = np.random.multivariate_normal(np.zeros(Kc), np.diag(covc), (N, T)) # temporal weights for new factors (N, T, Kb)
# noise info
signal = np.linalg.norm(A*np.sqrt(cova))**2 + np.linalg.norm(C*np.sqrt(covc))**2 
noisevar = signal/snr

print("signal power is %.3f"%(signal,))
print("noise power is %.3f"%(noisevar,))


signal power is 147.751
noise power is 1.478


In [123]:
# generate noise
noise = np.sqrt(noisevar)*np.random.randn(N, T, P)

# generate sample level data ( # subject x # time points x # regions)
X = np.dot(B, A.T) + noise
if Kc>0:
    X += np.dot(D, C.T)

# fitting

In [124]:
# Fit main components using regression
X_flat = np.reshape(X, (N*T, P))
print(X_flat.shape, A.shape)

B_est_flat,sserror ,_ ,_ = np.linalg.lstsq(A, X_flat.T)
print(B_est_flat.shape)
B_est = np.reshape(B_est_flat.T, (N, T, Ka))

R_est = X - np.dot(B_est, A.T)

relative_b_error = ((B-B_est)**2).sum()/((B)**2).sum()
relative_x_error = ((R_est)**2).sum()/((X)**2).sum()

print("relative b error is %.3f percent"%(relative_b_error*100.,))
print("relative residual error is %.3f percent"%(relative_x_error*100.,))


(10, 150) (150, 3)
(3, 10)
relative b error is 2.003 percent
relative residual error is 55.000 percent
